In [2]:
import torch
from torch import nn
from torchvision import models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MyMobileNetV2(nn.Module):
    def __init__(self, num_classes_fc=128, num_classes_output=2):
        super(MyMobileNetV2, self).__init__()

        # Load the pre-trained MobileNetV2 model
        mobilenet_v2 = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)

        # Modify the classifier for your specific task
        num_ftrs = mobilenet_v2.classifier[1].in_features # num_ftrs=1280
        mobilenet_v2.classifier[1] = nn.Linear(num_ftrs, num_classes_fc) # 3*224*224 = Image shape
        print(mobilenet_v2.classifier)

        # Fully connected network
        fully_connected_network = nn.Sequential(
            nn.Linear(num_classes_fc, num_classes_fc),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(num_classes_fc, num_classes_output)
        )

        # Combine MobileNetV2 and the fully connected network
        self.model = nn.Sequential(
            mobilenet_v2,
            # nn.AdaptiveAvgPool2d(1),
            # nn.Flatten(),
            fully_connected_network
        )

    def forward(self, x):
        return self.model(x)
model = MyMobileNetV2().to(device)

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=128, bias=True)
)


In [3]:
import torch
from torch import optim, nn
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from datasets import FaceAntiSpoofingDataset
from torch.utils.data import DataLoader, random_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transforms and other training configurations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the dataset
face_antispoofing_dataset = FaceAntiSpoofingDataset(root_dir='./datasets', celeba_split='train', lccfasd_split='training', transform=transform)

# Define the sizes of train and validation sets
train_size = int(0.8 * len(face_antispoofing_dataset))
val_size = len(face_antispoofing_dataset) - train_size

# Split the dataset
train_set, val_set = random_split(face_antispoofing_dataset, [train_size, val_size])


# Define training parameters
batch_size = 32
epochs = 10
learning_rate = 0.001

# Create data loaders
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# Initialize model and optimizer
model = MyMobileNetV2().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # print(inputs.shape)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")

    # Validation loop
    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs_val, labels_val in val_loader:
            inputs_val, labels_val = inputs_val.to(device), labels_val.to(device)

            outputs_val = model(inputs_val)
            _, predicted_val = torch.max(outputs_val.data, 1)

            total_val += labels_val.size(0)
            correct_val += (predicted_val == labels_val).sum().item()

        val_accuracy = correct_val / total_val
        print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Save the trained model
torch.save(model.state_dict(), 'face_anti_spoofing_model.pth')

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=128, bias=True)
)


KeyboardInterrupt: 